In [437]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt

import warnings 
warnings.filterwarnings('ignore')

sns.set_style("darkgrid")
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)

In [438]:
df0 = pd.read_csv("auto_scout_part3.csv")
df = df0.copy()

In [439]:
df.shape

(23466, 35)

In [440]:
df.sample(3)

,index,make_model,make,model,price,body_type,type,doors,origin_country,warranty,mileage,first_registration,gearbox,fuel_type,color,paint,seller,seats,engine_size,gears,drivetrain,cylinders,empty_weight,full_service_history,emission_class,upholstery_color,upholstery,previous_owner,horse_power,fuel_consumption,comfort_convenience,safety_security,entertainment,extras,current_location
22555,26922,Volvo XC60,Volvo,XC60,54890.0,Off-Road/Pick-up,Used,5.0,Germany,Yes,71000.0,2018.0,Automatic,Benzine,Grey,Metallic,Dealer,5.0,1.9,8.0,4WD,4.0,2394.000000,Yes,Euro 6,Black,Full Leather,1.0,230.0,2.10,Deluxe,Deluxe,Premium,Standard,DE
12862,14339,Skoda Kodiaq,Skoda,Kodiaq,45990.0,Off-Road/Pick-up,Used,5.0,Germany,Yes,28300.0,2020.0,Manual,Diesel,Grey,Metallic,Dealer,5.0,1.9,7.0,4WD,4.0,1895.000000,Yes,Under Class,Black,Full Leather,1.0,230.0,6.20,Deluxe,Premium,Premium,Standard,DE
14061,15711,Dacia Duster,Dacia,Duster,18500.0,Off-Road/Pick-up,Used,4.0,Spain,Yes,49481.0,2019.0,Manual,Benzine,Red,Metallic,Dealer,5.0,1.3,6.0,Front,4.0,1307.372263,Yes,Euro 6,Black,Cloth,1.0,122.0,6.05,Premium,Premium,Standard,Standard,DE


In [441]:
## Analyze Dataset in relation to target column

In [442]:
df.corr(numeric_only=True).price

index                -0.074571
price                 1.000000
doors                -0.002326
mileage              -0.397121
first_registration    0.378592
seats                -0.022777
engine_size           0.449031
gears                 0.238688
cylinders             0.431803
empty_weight          0.582551
previous_owner       -0.217414
horse_power           0.642658
fuel_consumption      0.272828
Name: price, dtype: float64

In [443]:
df.groupby('body_type')['price'].mean()

body_type
Compact             15942.252602
Convertible         20974.080620
Coupe               29398.671093
Off-Road/Pick-up    29776.944680
Sedan               23232.642793
Station wagon       22909.094669
Name: price, dtype: float64

In [444]:
df.groupby('type')['price'].mean()

type
Demonstration     45172.945113
Employee's car    38105.178641
Pre-registered    31303.228976
Used              21486.280803
Name: price, dtype: float64

In [445]:
df.groupby('origin_country')['price'].mean()

origin_country
Austria            18704.030769
Belgium            19817.017045
Bulgaria           41990.000000
Croatia            50985.000000
Czechia            29929.291667
Denmark            29183.611111
European Union     26969.880556
France             16788.827869
Germany            26943.262046
Hungary            35512.315789
Italy              16269.534629
Japan              32152.777778
Luxembourg         42833.125000
Malta              14690.000000
Mexico            250000.000000
Netherlands        10862.158809
Poland             24006.166667
Romania            18745.153846
Slovakia           31742.500000
Slovenia           31883.000000
Spain              19889.888768
Sweden             12196.666667
Switzerland        14063.333333
United States      35829.200000
Name: price, dtype: float64

In [446]:
df.groupby('warranty')['price'].mean()

warranty
No     21489.600265
Yes    25286.201877
Name: price, dtype: float64

In [447]:
df.groupby('gearbox')['price'].mean()

gearbox
Automatic    36409.303003
Manual       19716.627966
Name: price, dtype: float64

In [448]:
df.groupby('fuel_type')['price'].mean()

fuel_type
Benzine          24154.862592
Diesel           22244.761449
Electric         34137.948882
Hybrid           27957.368421
Hybrid/Diesel    52659.600000
Lpg              13650.466981
Others           22529.598485
Name: price, dtype: float64

In [449]:
df.loc[df[df.fuel_type == 'Hybrid/Diesel'].index, 'fuel_type'] = 'Hybrid'

In [450]:
df.groupby('fuel_type')['price'].mean()

fuel_type
Benzine     24154.862592
Diesel      22244.761449
Electric    34137.948882
Hybrid      28152.797468
Lpg         13650.466981
Others      22529.598485
Name: price, dtype: float64

In [451]:
df.loc[df[df.fuel_type == 'Others'].index, 'fuel_type'] = np.nan

In [452]:
df.fuel_type.isnull().sum()

264

In [453]:
df.fuel_type = df.groupby(['make', 'body_type'])['fuel_type'].transform(lambda x: x.fillna(x.mode()[0]))

In [454]:
df.fuel_type.isnull().sum()

0

In [455]:
df.fuel_type.value_counts()

fuel_type
Benzine     15031
Diesel       7257
Hybrid        653
Electric      313
Lpg           212
Name: count, dtype: int64

In [456]:
df.loc[df[(df.engine_size == 0.0) & ~(df.fuel_type == 'Electric')].index, 'engine_size'] = np.nan

In [457]:
df.engine_size = df.groupby(['make', 'fuel_type'])['engine_size'].transform(lambda x: x.fillna(x.mode()[0]))

In [458]:
df.engine_size.isnull().sum()

0

In [459]:
df.groupby('fuel_type')['price'].mean()

fuel_type
Benzine     24111.234249
Diesel      22286.765606
Electric    34137.948882
Hybrid      28053.781011
Lpg         13650.466981
Name: price, dtype: float64

In [460]:
df.groupby('color')['price'].mean()

color
Beige     15270.935323
Black     26158.425704
Blue      23031.492139
Bronze    16767.000000
Brown     18584.089710
Gold      24983.947368
Green     27137.888889
Grey      24686.835867
Orange    23332.216000
Red       21041.394191
Silver    21423.327508
Violet    12809.682927
White     22112.647482
Yellow    25187.655172
Name: price, dtype: float64

In [461]:
df.groupby('paint')['price'].mean()

paint
Metallic     25829.777400
Uni/basic    20942.286872
Name: price, dtype: float64

In [462]:
df.groupby('seller')['price'].mean()

seller
Dealer            24395.753924
Private seller    14597.695704
Name: price, dtype: float64

In [463]:
df.groupby('seats')['price'].mean()

seats
0.0      9400.000000
1.0      8625.000000
2.0     33396.856061
3.0     22732.347826
4.0     22874.786438
5.0     23073.194457
6.0     50284.875000
7.0     37767.156977
8.0     34362.780488
9.0     32399.257143
17.0    59990.000000
Name: price, dtype: float64

In [464]:
df[df.seats.isin([0, 17])].index

Index([3910, 3915, 4463, 10649, 10758, 11033, 12090, 12138, 13491, 13612,
       21179],
      dtype='int64')

In [465]:
df.drop(df[df.seats.isin([0, 17])].index, axis=0, inplace=True)

In [466]:
df.groupby('seats')['price'].mean()

seats
1.0     8625.000000
2.0    33396.856061
3.0    22732.347826
4.0    22874.786438
5.0    23073.194457
6.0    50284.875000
7.0    37767.156977
8.0    34362.780488
9.0    32399.257143
Name: price, dtype: float64

In [467]:
df.shape

(23455, 35)

In [468]:
df.groupby('gears')['price'].mean()

gears
1.0    31209.594904
4.0    10801.500000
5.0    14163.376392
6.0    21579.608281
7.0    33234.040000
8.0    44876.675506
Name: price, dtype: float64

In [469]:
df.groupby('drivetrain')['price'].mean()

drivetrain
4WD      45179.790181
Front    18729.598404
Rear     37798.718681
Name: price, dtype: float64

In [470]:
df.groupby('cylinders')['price'].mean() ## Use engine_size instead of cylinders

cylinders
0.0     34336.784946
1.0     38723.280576
2.0     16444.975904
3.0     17807.688354
4.0     21269.390349
5.0     13871.186472
6.0     50787.743567
7.0     85929.166667
8.0     75610.075342
11.0    71990.000000
12.0    65358.555556
16.0     1250.000000
Name: price, dtype: float64

In [471]:
df.groupby('full_service_history')['price'].mean()

full_service_history
No     20763.224444
Yes    26826.568346
Name: price, dtype: float64

In [472]:
df.groupby('emission_class')['price'].mean()

emission_class
Euro 1         16555.577778
Euro 2         13764.069231
Euro 3          7607.562212
Euro 4          8605.379933
Euro 5         13290.939244
Euro 6         29374.797810
Under Class    21985.114453
Name: price, dtype: float64

In [473]:
df.groupby('upholstery_color')['price'].mean()

upholstery_color
Beige     25482.069061
Black     23935.972555
Blue      15717.826087
Brown     35040.394619
Green     69284.300000
Grey      19579.287375
Orange    23273.742857
Other     23984.815418
Red       20591.840708
White     18065.078947
Yellow    21020.000000
Name: price, dtype: float64

In [474]:
df.groupby('upholstery')['price'].mean()

upholstery
Alcantara       35394.603905
Cloth           17578.568486
Full Leather    34262.107740
Other           22610.321364
Part Leather    25899.870117
Velour          17715.096970
Name: price, dtype: float64

In [475]:
df.groupby('comfort_convenience')['price'].mean()

comfort_convenience
Deluxe      41513.672671
Premium     23661.028015
Standard    15475.714540
Name: price, dtype: float64

In [476]:
df.groupby('safety_security')['price'].mean()

safety_security
Deluxe      44793.222919
Premium     25593.926053
Standard    16208.288493
Name: price, dtype: float64

In [477]:
df.groupby('entertainment')['price'].mean()

entertainment
Deluxe      57863.441948
Premium     30448.451067
Standard    17368.944949
Name: price, dtype: float64

In [478]:
df.groupby('extras')['price'].mean()

extras
Deluxe      42355.692308
Premium     36830.782946
Standard    21877.705785
Name: price, dtype: float64

In [479]:
df.groupby('current_location')['price'].mean()

current_location
AT    21249.054717
BE    24144.977626
BG    25500.000000
DE    28144.713659
ES    20891.705413
FR    21319.349352
IT    17341.858142
LU    22568.322581
NL    14368.233210
Name: price, dtype: float64

In [480]:
## Downsize Dataset

In [481]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23455 entries, 0 to 23465
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 23455 non-null  int64  
 1   make_model            23455 non-null  object 
 2   make                  23455 non-null  object 
 3   model                 23455 non-null  object 
 4   price                 23455 non-null  float64
 5   body_type             23455 non-null  object 
 6   type                  23455 non-null  object 
 7   doors                 23455 non-null  float64
 8   origin_country        23455 non-null  object 
 9   warranty              23455 non-null  object 
 10  mileage               23455 non-null  float64
 11  first_registration    23455 non-null  float64
 12  gearbox               23455 non-null  object 
 13  fuel_type             23455 non-null  object 
 14  color                 23455 non-null  object 
 15  paint                 23

In [482]:
df.corr(numeric_only=True).price

index                -0.074421
price                 1.000000
doors                -0.002322
mileage              -0.396970
first_registration    0.378433
seats                -0.027040
engine_size           0.462161
gears                 0.238624
cylinders             0.431863
empty_weight          0.582455
previous_owner       -0.217288
horse_power           0.642684
fuel_consumption      0.272785
Name: price, dtype: float64

In [483]:
df.drop(['index', 'doors', 'seats', 'cylinders'], axis=1, inplace=True)

In [484]:
df.drop(['current_location', 'upholstery_color'], axis=1, inplace=True)

In [485]:
df.shape

(23455, 29)

In [486]:
df.columns

Index(['make_model', 'make', 'model', 'price', 'body_type', 'type',
       'origin_country', 'warranty', 'mileage', 'first_registration',
       'gearbox', 'fuel_type', 'color', 'paint', 'seller', 'engine_size',
       'gears', 'drivetrain', 'empty_weight', 'full_service_history',
       'emission_class', 'upholstery', 'previous_owner', 'horse_power',
       'fuel_consumption', 'comfort_convenience', 'safety_security',
       'entertainment', 'extras'],
      dtype='object')

In [487]:
df.drop(['origin_country', 'paint',], axis=1, inplace=True)

In [488]:
df.drop(df[df.upholstery == 'Other'].index, axis=0, inplace=True)

In [489]:
df.groupby('upholstery')['price'].mean()

upholstery
Alcantara       35394.603905
Cloth           17578.568486
Full Leather    34262.107740
Part Leather    25899.870117
Velour          17715.096970
Name: price, dtype: float64

In [490]:
df.dtypes

make_model               object
make                     object
model                    object
price                   float64
body_type                object
type                     object
warranty                 object
mileage                 float64
first_registration      float64
gearbox                  object
fuel_type                object
color                    object
seller                   object
engine_size             float64
gears                   float64
drivetrain               object
empty_weight            float64
full_service_history     object
emission_class           object
upholstery               object
previous_owner          float64
horse_power             float64
fuel_consumption        float64
comfort_convenience      object
safety_security          object
entertainment            object
extras                   object
dtype: object

In [491]:
df['make_model'] = df.make + ' ' + df.model

In [492]:
df.drop('model', axis=1, inplace=True)

In [493]:
df.sample()

,make_model,make,price,body_type,type,warranty,mileage,first_registration,gearbox,fuel_type,color,seller,engine_size,gears,drivetrain,empty_weight,full_service_history,emission_class,upholstery,previous_owner,horse_power,fuel_consumption,comfort_convenience,safety_security,entertainment,extras
13286,Skoda Superb,Skoda,35990.0,Sedan,Used,Yes,67649.0,2020.0,Manual,Benzine,Blue,Dealer,1.3,6.0,Front,1730.0,Yes,Euro 6,Cloth,1.0,122.0,1.4,Deluxe,Deluxe,Premium,Standard


In [494]:
df.columns

Index(['make_model', 'make', 'price', 'body_type', 'type', 'warranty',
       'mileage', 'first_registration', 'gearbox', 'fuel_type', 'color',
       'seller', 'engine_size', 'gears', 'drivetrain', 'empty_weight',
       'full_service_history', 'emission_class', 'upholstery',
       'previous_owner', 'horse_power', 'fuel_consumption',
       'comfort_convenience', 'safety_security', 'entertainment', 'extras'],
      dtype='object')

In [495]:
df = df.reset_index(drop=True)

In [496]:
df.index

RangeIndex(start=0, stop=22898, step=1)

In [497]:
df.to_csv('auto_scout_part4.csv', index=False)

In [498]:
dfx = pd.read_csv('auto_scout_part4.csv')

In [563]:
## Dummy operations

In [565]:
df['make_model'] = df.make_model.factorize()[0] + 1

In [569]:
df.body_type.value_counts()

body_type
Off-Road/Pick-up    4617
Station wagon       4415
Sedan               4333
Compact             4120
Coupe               2889
Convertible         2524
Name: count, dtype: int64

In [571]:
df['body_type'] = df.body_type.map({'Off-Road/Pick-up': 2, 'Station wagon': 1, 'Sedan': 0, 'Compact': 3, 'Coupe': 4, 'Convertible': 5})

In [573]:
df['type'] = df.type.map({'Used': 0, 'Demonstration': 1, 'Pre-registered': 2, "Employee's car": 3})

In [575]:
df['warranty'] = df.warranty.factorize()[0]

In [577]:
df['gearbox'] = df.gearbox.map({'Manual': 0, 'Automatic': 1})

In [579]:
df.fuel_type.value_counts()

fuel_type
Benzine     14673
Diesel       7087
Hybrid        642
Electric      297
Lpg           199
Name: count, dtype: int64

In [581]:
df['fuel_type'] = df.fuel_type.map({'Benzine': 0, 'Diesel': 1, 'Hybrid': 2, 'Electric': 3, 'Lpg': 4})

In [585]:
df['color'] = df.color.factorize()[0] + 1

In [587]:
df['seller'] = df.seller.map({'Dealer': 0, 'Private seller': 1})

In [589]:
df.drop(df[df.gears == 4.0].index, axis=0, inplace=True)

In [591]:
df.gears.value_counts()

gears
6.0    12467
5.0     5244
8.0     2675
7.0     1667
1.0      759
Name: count, dtype: int64

In [593]:
df.shape

(22812, 26)

In [595]:
df.drivetrain.value_counts()

drivetrain
Front    18104
4WD       3367
Rear      1341
Name: count, dtype: int64

In [597]:
df['drivetrain'] = df.drivetrain.map({'Front': 0, '4WD': 1, 'Rear': 2})

In [599]:
df['full_service_history'] = df.full_service_history.map({'No': 0, 'Yes': 1})

In [601]:
df.emission_class.value_counts()

emission_class
Euro 6         11457
Under Class     7140
Euro 5          2184
Euro 4          1447
Euro 3           422
Euro 2           122
Euro 1            40
Name: count, dtype: int64

In [603]:
df['emission_class'] = df.emission_class.map({
    'Under Class': 6,
    'Euro 1': 1,
    'Euro 2': 2,
    'Euro 3': 3,
    'Euro 4': 4,
    'Euro 5': 5,
    'Euro 6': 0
})

In [605]:
df.upholstery.value_counts()

upholstery
Cloth           12727
Full Leather     6070
Part Leather     3138
Alcantara         715
Velour            162
Name: count, dtype: int64

In [607]:
df['upholstery'] = df.upholstery.map({
    'Cloth': 0,
    'Velour': 4,
    'Part Leather': 2,
    'Full Leather': 1,
    'Alcantara': 3
})

In [609]:
df['comfort_convenience'] = df.comfort_convenience.map({'Standard': 0, 'Premium': 1, 'Deluxe': 2})

In [611]:
df['safety_security'] = df.safety_security.map({'Standard': 0, 'Premium': 1, 'Deluxe': 2})

In [613]:
df['entertainment'] = df.entertainment.map({'Standard': 0, 'Premium': 1, 'Deluxe': 2})

In [615]:
df['extras'] = df.extras.map({'Standard': 0, 'Premium': 1, 'Deluxe': 2})

In [617]:
df.shape

(22812, 26)

In [619]:
df.dtypes

make_model                int64
make                     object
price                   float64
body_type                 int64
type                      int64
warranty                  int64
mileage                 float64
first_registration      float64
gearbox                   int64
fuel_type                 int64
color                     int64
seller                    int64
engine_size             float64
gears                   float64
drivetrain                int64
empty_weight            float64
full_service_history      int64
emission_class            int64
upholstery                int64
previous_owner          float64
horse_power             float64
fuel_consumption        float64
comfort_convenience       int64
safety_security           int64
entertainment             int64
extras                    int64
dtype: object

In [627]:
df[pd.get_dummies(df.make, dtype='float').columns] = pd.get_dummies(df.make, dtype='float')

In [631]:
df.drop('make', axis=1, inplace=True)

In [633]:
df.sample()

,make_model,price,body_type,type,warranty,mileage,first_registration,gearbox,fuel_type,color,seller,engine_size,gears,drivetrain,empty_weight,full_service_history,emission_class,upholstery,previous_owner,horse_power,fuel_consumption,comfort_convenience,safety_security,entertainment,extras,Dacia,Fiat,Ford,Hyundai,Mercedes-Benz,Nissan,Opel,Peugeot,Renault,SEAT,Skoda,Toyota,Volvo
3311,207,33900.0,2,1,1,2050.0,2020.0,0,0,4,0,1.5,8.0,1,1875.0,1,0,0,1.0,230.0,1.5,1,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [635]:
df = df.astype('float')

In [637]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22812 entries, 0 to 22897
Data columns (total 38 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   make_model            22812 non-null  float64
 1   price                 22812 non-null  float64
 2   body_type             22812 non-null  float64
 3   type                  22812 non-null  float64
 4   warranty              22812 non-null  float64
 5   mileage               22812 non-null  float64
 6   first_registration    22812 non-null  float64
 7   gearbox               22812 non-null  float64
 8   fuel_type             22812 non-null  float64
 9   color                 22812 non-null  float64
 10  seller                22812 non-null  float64
 11  engine_size           22812 non-null  float64
 12  gears                 22812 non-null  float64
 13  drivetrain            22812 non-null  float64
 14  empty_weight          22812 non-null  float64
 15  full_service_history  22

In [641]:
df.corr().price

make_model             -0.164978
price                   1.000000
body_type              -0.015360
type                    0.207731
warranty                0.083429
mileage                -0.396836
first_registration      0.377660
gearbox                 0.324500
fuel_type              -0.011603
color                  -0.074384
seller                 -0.115549
engine_size             0.463492
gears                   0.239930
drivetrain              0.391300
empty_weight            0.581780
full_service_history    0.134816
emission_class         -0.213923
upholstery              0.224692
previous_owner         -0.217046
horse_power             0.642935
fuel_consumption        0.275761
comfort_convenience     0.397864
safety_security         0.355846
entertainment           0.363108
extras                  0.220328
Dacia                  -0.101309
Fiat                   -0.103428
Ford                    0.087652
Hyundai                -0.050131
Mercedes-Benz           0.430213
Nissan    

In [647]:
df.to_csv('auto_scout_final.csv', index=False)